In [1]:
import pandas as pd
import numpy as np
import requests
import json
import unicodedata
from shapely.geometry import shape
from scipy.sparse.csgraph import dijkstra
import time

In [10]:
# Configurações iniciais
ORS_API_KEY = '5b3ce3597851110001cf6248aacd4755cfdd453e8b72af7e913c7fad'  # Chave da API OpenRouteService
MAX_DESTINOS = 40  # Não sobrecarregar o OpenRouteService
SLEEP_TIME = 0.5

In [3]:
def normalizar(texto):
    """Remove acentos, coloca o texto em minúsculo e tira espaços extras."""
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8').lower().strip()

In [4]:
def geocodificar_origem(endereco, api_key):
    """Converte endereço para latitude/longitude usando OpenRouteService."""
    url = 'https://api.openrouteservice.org/geocode/search'
    params = {
        'api_key': api_key,
        'text': endereco,
        'boundary.country': 'BR'
    }
    response = requests.get(url, params=params)
    data = response.json()

    if 'features' not in data or len(data['features']) == 0:
        raise ValueError("Endereço não encontrado.")

    coords = data['features'][0]['geometry']['coordinates']
    # OpenRouteService retorna [lon, lat]
    return f"{coords[1]},{coords[0]}"  # lat, lon

In [5]:
def carregar_postos_soro(caminho_csv, soro_necessario):
    """Carrega os postos e filtra pelo soro necessário."""
    df = pd.read_csv(caminho_csv)
    df = df[df['Tipos de Soro'].str.contains(soro_necessario, case=False, na=False)]
    return df

In [6]:
def calcular_distancias_com_blocos(df, origem_coords, modo, api_key):
    """Calcula distância entre a origem e os postos usando OpenRouteService Matrix API."""
    df = df.copy()
    df['Distância (km)'] = None
    df['Tempo estimado (min)'] = None

    lat_origem, lon_origem = map(float, origem_coords.split(','))

    # Modo para a API
    modos_ors = {
        'driving': 'driving-car',
        'walking': 'foot-walking',
        'bicycling': 'cycling-regular',
    }
    modo_ors = modos_ors.get(modo, 'driving-car')

    for i in range(0, len(df), MAX_DESTINOS):
        bloco = df.iloc[i:i+MAX_DESTINOS].copy()

        # Origem + destinos
        locations = [[lon_origem, lat_origem]] + bloco[['longitude', 'latitude']].values.tolist()

        url = "https://api.openrouteservice.org/v2/matrix/" + modo_ors
        headers = {
            'Authorization': api_key,
            'Content-Type': 'application/json'
        }
        payload = {
            "locations": locations,
            "metrics": ["distance", "duration"],
            "units": "km",
            "sources": [0],  # Origem é o índice 0
            "destinations": list(range(1, len(locations)))  # Destinos são 1 até N
        }

        response = requests.post(url, headers=headers, json=payload)
        data = response.json()

        if 'distances' in data:
            distancias = data['distances'][0]  # Primeira linha: da origem para todos os destinos
            tempos = [t/60 for t in data['durations'][0]]  # converter segundos para minutos
            df.loc[bloco.index, 'Distância (km)'] = distancias
            df.loc[bloco.index, 'Tempo estimado (min)'] = tempos
        else:
            print("Erro na resposta ORS:", data)

        time.sleep(SLEEP_TIME)

    return df

In [7]:
def mostrar_resultado(df):
    """Mostra o posto de saúde mais próximo."""
    if df['Distância (km)'].notna().any():
        mais_proxima = df.loc[df['Distância (km)'].idxmin()]

        print("\n" + "="*50)
        print("🔎 RESULTADO - Unidade de Saúde Mais Próxima".center(50))
        print("="*50)
        print(f"🏥 Unidade:    {mais_proxima['Unidade de Saúde']}")
        print(f"📍 Endereço:   {mais_proxima['Endereço']}")
        print(f"🏙️  Cidade:     {mais_proxima['Cidade']}")
        print(f"📞 Telefone:   {mais_proxima['Telefone']}")
        print(f"🛣️  Distância:  {mais_proxima['Distância (km)']:.2f} km")
        print(f"⏱️  Tempo:      {mais_proxima['Tempo estimado (min)']:.1f} minutos")
        print("="*50 + "\n")
    else:
        print("Nenhuma distância válida foi calculada.")

In [11]:
# Carregamento do arquivo GeoJSON contendo as cidades do estado de SP
print("Carregando GeoJSON...")
with open('geojson_sp.json', 'r', encoding='utf-8') as f:
    geojson = json.load(f)

# Extração das cidades e suas coordenadas
cidades = []
coordenadas = []
for feature in geojson['features']:
    cidades.append(feature['properties']['name'])
    centroide = shape(feature['geometry']).centroid
    coordenadas.append((centroide.y, centroide.x))  # Latitude, Longitude

# Construção da matriz de distâncias aproximada (distância Euclidiana)
n = len(cidades)
matriz = np.full((n, n), np.inf)
for i in range(n):
    matriz[i, i] = 0.0

for i in range(n):
    for j in range(i+1, n):
        matriz[i, j] = matriz[j, i] = np.linalg.norm(np.subtract(coordenadas[i], coordenadas[j]))

df_matriz = pd.DataFrame(matriz, index=cidades, columns=cidades)

# Pergunta endereço completo
endereco_origem = input("Digite o endereço de origem (Rua, Número, Cidade, Estado): ")
origem_coords = geocodificar_origem(endereco_origem, ORS_API_KEY)

# Detecta a cidade mais próxima
lat_origem, lon_origem = map(float, origem_coords.split(','))
dist_cidades = [np.linalg.norm(np.subtract((lat_origem, lon_origem), (lat, lon))) for lat, lon in coordenadas]
cidade_origem = cidades[np.argmin(dist_cidades)]

print(f"📍 Cidade identificada: {cidade_origem}")

entrada_usuario_normalizada = normalizar(cidade_origem)
cidades_normalizadas = [normalizar(c) for c in cidades]

origem_idx = cidades_normalizadas.index(entrada_usuario_normalizada)
distancias, _ = dijkstra(matriz, directed=False, indices=origem_idx, return_predecessors=True)

# Seleciona 24 cidades mais próximas + a própria
resultado = pd.DataFrame({
    'Cidade': cidades,
    'Distância (km)': distancias
}).sort_values(by='Distância (km)')

cidades_proximas = resultado['Cidade'].head(25).tolist()

# Pergunta animal causador
animal = normalizar(input("Qual animal causou o acidente? "))

mapa_soros_animais = {
    "escorpiao": "ESCORPIÔNICO",
    "aranha marrom": "ARACNÍDICO",
    "aranha armadeira": "ARACNÍDICO",
    "aranha viuva-negra": "ARACNÍDICO",
    "lonomia": "LONÔMICO",
    "taturana": "LONÔMICO",
    "cobra jararaca": "BOTRÓPICO",
    "cobra surucucu": "LAQUÉTICO",
    "cobra cascavel": "CROTÁLICO",
    "cobra coral": "ELAPÍDICO",
}

soro_necessario = mapa_soros_animais.get(animal)
if not soro_necessario:
    print("Animal não reconhecido.")
    exit()

# Carrega e filtra postos
df_postos = carregar_postos_soro('postos_geolocalizados.csv', soro_necessario)
df_postos['Cidade'] = df_postos['Cidade'].apply(normalizar)
cidades_proximas = [normalizar(cidade) for cidade in cidades_proximas]
postos_filtrados = df_postos[df_postos['Cidade'].isin(cidades_proximas)]

# Escolha modo de transporte
modo_usuario = normalizar(input("Escolha o modo de transporte (carro, caminhando, bicicleta): "))
mapa_modos = {'carro': 'driving', 'caminhando': 'walking', 'bicicleta': 'bicycling'}
modo_api = mapa_modos.get(modo_usuario, 'driving')

# Calcula distâncias reais
postos_resultado = calcular_distancias_com_blocos(postos_filtrados, origem_coords, modo_api, ORS_API_KEY)

postos_resultado = postos_resultado.copy()
postos_resultado['Distância (km)'] = pd.to_numeric(postos_resultado['Distância (km)'], errors='coerce')

postos_validos = postos_resultado.dropna(subset=['Distância (km)'])

if postos_validos.empty:
    print("❌ Nenhum posto com distância válida encontrada.")
    exit()

mostrar_resultado(postos_validos)

Carregando GeoJSON...
📍 Cidade identificada: São Carlos

   🔎 RESULTADO - Unidade de Saúde Mais Próxima    
🏥 Unidade:    IRMANDADE SANTA CASA
📍 Endereço:   R. PAULINO BOTELHO DE ABREU SAMPAIO, 573
🏙️  Cidade:     sao carlos
📞 Telefone:   (16) 3509-1100
🛣️  Distância:  2.41 km
⏱️  Tempo:      4.8 minutos



In [ ]:
df_10 = postos_validos.nsmallest(10, 'Distância (km)')
df_10[['Unidade de Saúde', 'Endereço', 'Cidade', 'Telefone', 'Tipos de Soro', 'Distância (km)', 'Tempo estimado (min)']].head(10)

,Unidade de Saúde,Endereço,Cidade,Telefone,Tipos de Soro,Distância (km),Tempo estimado (min)
187,HOSPITAL GUILHERME ÁLVARO - SANTOS,R. OSWALDO CRUZ Nº197,santos,(13) 3202-1300,"BOTRÓPICO, CROTÁLICO, ELAPÍDICO, ESCORPIÔNICO,...",3.40,4.937667
63,HOSP. SANTO AMARO - GUARUJÁ,R. QUINTO BERTOLDI Nº40 V. MAIA,guaruja,(13) 3389-1515,"BOTRÓPICO, ARACNÍDICO E ESCORPIÔNICO.",12.89,22.214667
62,P.S. CENTRAL GUIOMAR FERREIRA ROEBBELEN - CUBATÃO,AV. 9 DE ABRIL S/Nº,cubatao,(13) 3361-8786,"BOTRÓPICO, ARACNÍDICO E ESCORPIÔNICO.",17.56,17.970667
66,PS QUIETUDE,"AV. MINISTRO MARCOS FREIRE, S/N, JD. QUIETUDE",praia grande,(13) 3471-4221,"BOTRÓPICO, CROTÁLICO, ARACNÍDICO E ESCORPIÔNICO.",19.72,24.1595
65,UPA MONGAGUÁ,"AV. MONTEIRO LOBATO, 9.400 – AGENOR DE CAMPOS",mongagua,(13) 3507-1110,"BOTRÓPICO, CROTÁLICO, ARACNÍDICO E ESCORPIÔNICO.",45.09,48.369667
61,HOSPITAL MUNICIPAL DE BERTIOGA,PÇA. VICENTE MOLINARI S/Nº,bertioga,(13) 3319-9040,"BOTRÓPICO, ARACNÍDICO E ESCORPIÔNICO.",46.30,59.779167
101,HOSPITAL DE URGÊNCIA - S.B. DO CAMPO,"R. JOAQUIM NABUCO, 380 - CENTRO, CEP: 09720-375",sao bernardo do campo,(11) 2630-9827 / 9824 /966051801,ESCORPIÔNICO,49.39,49.3745
224,HOSPITAL MUNICIPAL DE PARELHEIROS,"RUA EUZÉBIO COGHI, 841, JARDIM ROSCHEL, SÃO PA...",sao paulo,(11) 4673-9660,"BOTRÓPICO, CROTÁLICO, ELAPÍDICO, ESCORPIÔNICO ...",64.33,75.749333
2,HOSPITAL ESTADUAL PROF. LIBERATO JOHN ALPHONSE...,"R. FRANCISCO OCTAVIO PACCA, 180, PARQUE DAS NA...",sao paulo,(11) 3544-9444 RAMAL 415/218 / 11-947118758,ESCORPIÔNICO,70.77,66.272667
0,HOSPITAL VITAL BRAZIL DO INSTITUTO BUTANTAN,AV. VITAL BRASIL Nº 1500 - INSTITUTO BUTANTAN,sao paulo,(11) 2627-9529,"BOTRÓPICO, CROTÁLICO, ELAPÍDICO, LAQUÉTICO, ES...",81.64,75.640667
